# General Setup

In [75]:
import json
import pandas as pd
import subprocess
import os

In [76]:
with open('data/curated/player_data.json', 'r') as f:
	player_data = json.load(f)

In [77]:
df = pd.DataFrame(player_data)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15530 entries, 0 to 15529
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15530 non-null  int64  
 1   match_start_time         15530 non-null  int64  
 2   account_id               15530 non-null  int64  
 3   name                     15530 non-null  object 
 4   fantasy_role             15530 non-null  object 
 5   hero_id                  15530 non-null  int64  
 6   hero_variant             15530 non-null  object 
 7   isRadiant                15530 non-null  bool   
 8   radiant_team_id          15530 non-null  int64  
 9   dire_team_id             15530 non-null  int64  
 10  tournament_start_date    15530 non-null  object 
 11  duration                 15530 non-null  int64  
 12  win                      15530 non-null  int64  
 13  kills                    15530 non-null  int64  
 14  deaths                   15

In [79]:
df = df.astype({
    'match_id': 'string',
    'account_id': 'string',
    'hero_id': 'string',
    'radiant_team_id': 'string',
    'dire_team_id': 'string',
    'win': 'boolean',
    'firstblood_claimed': 'boolean',
    'team_id': 'string',
	'enemy_team_id': 'string',
    'valveId': 'string',
})

In [80]:
q1_glicko = df['glicko2_rating'].quantile(0.25)
df['glicko2_rating'] = df['glicko2_rating'].fillna(q1_glicko)
q1_enemy_glicko = df['enemy_glicko2_rating'].quantile(0.25)
df['enemy_glicko2_rating'] = df['enemy_glicko2_rating'].fillna(q1_enemy_glicko)

In [81]:
mask = df['rating_advantage'].isna()
df.loc[mask, 'rating_advantage'] = df.loc[mask, 'glicko2_rating'] - df.loc[mask, 'enemy_glicko2_rating']

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15530 entries, 0 to 15529
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15530 non-null  string 
 1   match_start_time         15530 non-null  int64  
 2   account_id               15530 non-null  string 
 3   name                     15530 non-null  object 
 4   fantasy_role             15530 non-null  object 
 5   hero_id                  15530 non-null  string 
 6   hero_variant             15530 non-null  object 
 7   isRadiant                15530 non-null  bool   
 8   radiant_team_id          15530 non-null  string 
 9   dire_team_id             15530 non-null  string 
 10  tournament_start_date    15530 non-null  object 
 11  duration                 15530 non-null  int64  
 12  win                      15530 non-null  boolean
 13  kills                    15530 non-null  int64  
 14  deaths                   15

# XGBoost
Without time dimension.

In [83]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error


In [84]:
not_features = [
	'match_id', 'name', 'match_start_time', 'radiant_team_id', 'dire_team_id',
	'tournament_start_date', 'teamName', 'valveId', 'enemy_teamName', 'enemy_valveId', 'fantasy_score'
]

In [85]:
X = df.drop(columns=not_features)
y = df['fantasy_score']

In [86]:
X = pd.get_dummies(X, columns=['fantasy_role', 'hero_id', 'hero_variant', 'account_id', 'team_id', 'enemy_team_id'])


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1001)


In [88]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [89]:
y_pred = model.predict(X_test)
print("RMSE:", root_mean_squared_error(y_test, y_pred))

RMSE: 0.8458655490464495


In [90]:
df[['team_id', 'enemy_team_id']].drop_duplicates()

,team_id,enemy_team_id
0,8599101,2163
5,2163,8599101
30,8599101,8291895
35,8291895,8599101
60,9247354,8291895
...,...,...
15455,9131584,8599101
15460,9247354,8254145
15465,8254145,9247354
15490,9828954,8597976


In [91]:
def build_prediction_rows(df, matchups, lineup_dir, rolling_n=5):
    rows = []
    for team_id, enemy_team_id in matchups:
        lineup_path = os.path.join(lineup_dir, f"team_lineup_{team_id}.json")
        if not os.path.exists(lineup_path):
            subprocess.run(["python", "get_current_lineups.py", str(team_id)])
        if not os.path.exists(lineup_path):
            continue  # Skip if still missing
        with open(lineup_path, "r", encoding="utf-8") as f:
            lineup = [p for p in json.load(f) if p.get("is_current_team_member")]
        for player in lineup:
            account_id = str(player["account_id"])
            player_hist = df[df["account_id"] == account_id].sort_values("match_start_time")
            if player_hist.empty:
                continue
            numeric_cols = player_hist.select_dtypes(include='number').columns.difference(
                ['match_id', 'account_id', 'radiant_team_id', 'dire_team_id', 'team_id', 'enemy_team_id', 'match_start_time']
            )
            rolling = player_hist[numeric_cols].tail(rolling_n).mean()
            row = player_hist.iloc[-1].copy()
            row[numeric_cols] = rolling
            row["team_id"] = team_id
            row["enemy_team_id"] = enemy_team_id
            rows.append(row)
    return pd.DataFrame(rows)

In [92]:
def predict_players_for_matchups(df, model, matchup_list, X, not_features, lineup_dir="data/lineups", rolling_n=5):
    # Ensure all lineup files exist (call API if not)
    team_ids = set()
    for t1, t2 in matchup_list:
        team_ids.add(t1)
        team_ids.add(t2)
    for team_id in team_ids:
        lineup_path = os.path.join(lineup_dir, f"team_lineup_{team_id}.json")
        if not os.path.exists(lineup_path):
            subprocess.run(["python", "get_current_lineups.py", str(team_id)])
    
    # Build synthetic prediction rows
    pred_df = build_prediction_rows(df, matchup_list, lineup_dir, rolling_n=rolling_n)
    if pred_df.empty:
        print("No eligible players found for the given matchups.")
        return pd.DataFrame()
    # Prepare features (same as training)
    X_pred = pred_df.drop(columns=not_features)
    X_pred = pd.get_dummies(X_pred, columns=['fantasy_role', 'hero_id', 'hero_variant', 'account_id', 'team_id', 'enemy_team_id'])
    X_pred = X_pred.reindex(columns=X.columns, fill_value=0)
    # Predict fantasy scores
    pred_df['predicted_fantasy_score'] = model.predict(X_pred)
    # Output: account_id, name, fantasy_role, team_id, enemy_team_id, predicted_fantasy_score
    return pred_df[['account_id', 'name', 'fantasy_role', 'team_id', 'enemy_team_id', 'predicted_fantasy_score']]

In [97]:
def make_symmetric_matchups(matchups):
    symmetric = []
    for t1, t2 in matchups:
        symmetric.append((t1, t2))
        symmetric.append((t2, t1))
    return symmetric

In [98]:
matchups = [('7119388', '8261500'), ('8597976', '7119388')] # Spirit vs. Xtreme Gaming, Spirit vs. Talon
symetric_matchups = make_symmetric_matchups(matchups)
result = predict_players_for_matchups(df, model, symetric_matchups, X, not_features)

In [99]:
result

,account_id,name,fantasy_role,team_id,enemy_team_id,predicted_fantasy_score
14641,113331514,Miposhka,Support,7119388,8261500,13.003644
14640,321580662,Yatoro,Core,7119388,8261500,19.310001
14644,302214028,Collapse,Core,7119388,8261500,15.361670
14642,106305042,Larl,Mid,7119388,8261500,15.639341
14643,847565596,rue,Support,7119388,8261500,16.039099
6689,203351055,Malik,Core,7119388,8261500,9.571531
14896,137129583,Xm,Mid,8261500,7119388,13.757675
14898,157475523,XinQ,Support,8261500,7119388,13.231318
14897,129958758,Xxs,Core,8261500,7119388,11.340541
14895,898754153,Ame,Core,8261500,7119388,18.129608
